Importanción de librerias

In [31]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

### Parte 1: Procesamiento de los datos

Cargando de los datos

In [33]:
dataset = pd.read_csv('datos_crudos/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Separando las variables independientes y la variable dependiente

In [34]:
X = dataset.iloc[:, 3:13] #Variables independientes
y = dataset.iloc[:, 13] #Variable dependiente

Codificación de datos categóricos

In [35]:
#Codificando la variable categórica "Geography"
one_hot_encoding = pd.get_dummies(X['Geography'], prefix='Geography').astype(int)
X = pd.concat([X, one_hot_encoding], axis=1)
#Se elimina la columna "Geography_Spain" para evitar problemas de multicolinealidad
X.drop(['Geography', 'Geography_Spain'], axis=1, inplace=True)
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany
0,619,Female,42,2,0.00,1,1,1,101348.88,1,0
1,608,Female,41,1,83807.86,1,0,1,112542.58,0,0
2,502,Female,42,8,159660.80,3,1,0,113931.57,1,0
3,699,Female,39,1,0.00,2,0,0,93826.63,1,0
4,850,Female,43,2,125510.82,1,1,1,79084.10,0,0


In [36]:
#Codificando la variable categórica "Gender"
sexo = pd.get_dummies(X['Gender'], prefix='sex').astype(int)
X = pd.concat([X, sexo], axis=1)
X.drop(['Gender', 'sex_Male'], axis=1, inplace=True)
X.rename(columns={'sex_Female':'Gender'}, inplace=True)
X.head()
#La columna Gender 1 es para femenino y 0 para masculino

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Gender
0,619,42,2,0.00,1,1,1,101348.88,1,0,1
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,1
3,699,39,1,0.00,2,0,0,93826.63,1,0,1
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1


Dividisión del dataframe en entrenamiento y en prueba

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Escalando las variables

In [38]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Parte 2: Construcción de la red neuronal artificial (RNA)

Inicializando la RNA

In [39]:
#Se crea la RNA
clasificador = Sequential()

#Se agregan las capas ocultas
clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

#Se agrega la capa de salidas
clasificador.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

#Compilar la RNA
clasificador.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics = ["accuracy"]
)

Entrenando la RNA

In [40]:
clasificador.fit(
    x=X_train_scaled,
    y=y_train,
    batch_size=10,
    epochs=50
)

Epoch 1/50


800/800 [==============================] - 5s 4ms/step - loss: 0.4949 - accuracy: 0.7961
Epoch 2/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4318 - accuracy: 0.7960
Epoch 3/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4290 - accuracy: 0.7960
Epoch 4/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4255 - accuracy: 0.7960
Epoch 5/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4213 - accuracy: 0.8040
Epoch 6/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4183 - accuracy: 0.8230
Epoch 7/50
800/800 [==============================] - 3s 3ms/step - loss: 0.4154 - accuracy: 0.8288
Epoch 8/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4145 - accuracy: 0.8286
Epoch 9/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4128 - accuracy: 0.8319
Epoch 10/50
800/800 [==============================] - 2s 3ms/step - loss: 0.4110 - accuracy: 0.8320
Epoch 11/5

### Parte 3: Evaluación de la RNA

Se realiza las predicciones

In [41]:
y_pred = clasificador.predict(x=X_test_scaled)
y_pred = (y_pred > 0.5)

63/63 [==============================] - 1s 4ms/step


Se crea la matriz de confusión

In [42]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1543,   52],
       [ 260,  145]], dtype=int64)

### Parte 4: Predecir una nueva observación

Utilizando el modelo de RNA para predecir si el cliente con la siguiente información abandonará el banco:

- Geografia: Francia

- Puntaje de crédito: 600

- Género masculino

- Edad: 40 años de edad

- Tenencia: 3 años.

- Saldo: $ 60000

- Número de productos: 2

- ¿Este cliente tiene una tarjeta de crédito? Sí

- ¿Es este cliente un miembro activo? Sí

- Salario estimado: $ 50000

In [43]:
obs = np.array([[600, 40, 3, 60000, 2, 1, 1, 50000, 1, 0, 0]])
obs_scaled = scaler.transform(obs)
nueva_pred = clasificador.predict(obs_scaled)
print(nueva_pred > 0.5)

1/1 [==============================] - 0s 83ms/step
[[False]]


### Parte 5: Validación cruzada

Se define la función se crea el clasificador

In [44]:
def construir_clasificador():
    #Se crea la RNA
    clasificador = Sequential()

    #Se agregan las capas ocultas
    clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))
    clasificador.add(Dropout(rate=0.1))
    clasificador.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    clasificador.add(Dropout(rate=0.1))
    
    #Se agrega la capa de salidas
    clasificador.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    #Compilar la RNA
    clasificador.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics = ["accuracy"]
    )

    return clasificador

Se crea el clasificador en  el contexto de sklearn

In [45]:
clasificador = KerasClassifier(build_fn = construir_clasificador, batch_size = 10, nb_epoch = 40)
accuracies = cross_val_score(estimator=clasificador, X=X_train_scaled, y=y_train, cv=10, verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


80/80 [==============================] - 0s 2ms/step - loss: 0.4231 - accuracy: 0.7950


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   45.5s finished


Resultados de la validación cruzada

In [46]:
print('Resultados de la validación cruzada')
print('Sesgo: {:.2f}'.format(accuracies.mean()))
print('Varianza: {:.2f}'.format(accuracies.std()))

Resultados de la validación cruzada
Sesgo: 0.80
Varianza: 0.01
